<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Dimensionality-reduction" data-toc-modified-id="Dimensionality-reduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Dimensionality reduction</a></span><ul class="toc-item"><li><span><a href="#PCA" data-toc-modified-id="PCA-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>PCA</a></span></li><li><span><a href="#t-SNE" data-toc-modified-id="t-SNE-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>t-SNE</a></span></li><li><span><a href="#More-control-on-t-SNE-parameters" data-toc-modified-id="More-control-on-t-SNE-parameters-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>More control on t-SNE parameters</a></span><ul class="toc-item"><li><span><a href="#Dimension-of-the-map-space" data-toc-modified-id="Dimension-of-the-map-space-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Dimension of the map space</a></span></li><li><span><a href="#Perplexity" data-toc-modified-id="Perplexity-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Perplexity</a></span></li><li><span><a href="#Early-exaggeration" data-toc-modified-id="Early-exaggeration-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Early exaggeration</a></span></li><li><span><a href="#Learning-rate" data-toc-modified-id="Learning-rate-1.3.4"><span class="toc-item-num">1.3.4&nbsp;&nbsp;</span>Learning rate</a></span></li><li><span><a href="#Number-of-iterations" data-toc-modified-id="Number-of-iterations-1.3.5"><span class="toc-item-num">1.3.5&nbsp;&nbsp;</span>Number of iterations</a></span></li><li><span><a href="#Approximated-vs.-exact-execution" data-toc-modified-id="Approximated-vs.-exact-execution-1.3.6"><span class="toc-item-num">1.3.6&nbsp;&nbsp;</span>Approximated vs. exact execution</a></span></li></ul></li><li><span><a href="#Jointly-using-PCA-and-t-SNE" data-toc-modified-id="Jointly-using-PCA-and-t-SNE-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Jointly using PCA and t-SNE</a></span></li><li><span><a href="#An-example:-word2vec" data-toc-modified-id="An-example:-word2vec-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>An example: word2vec</a></span></li></ul></li></ul></div>

# Dimensionality reduction

Sources:

- J. Leskovec, A. Rajaraman, J. Ullman, [Mining of massive datasets](http://www.mmds.org/), chapter 11,
- C. Rossant, [An illustrated introduction to the t-SNE algorithm](https://www.oreilly.com/learning/an-illustrated-introduction-to-the-t-sne-algorithm),
- M. Pathak, [Introduction to t-SNE](https://www.datacamp.com/community/tutorials/introduction-t-sne).

Supplementary material:

- L. van der Meerten, G. Hinton, Journal of Machine Learning Research 9 (2008) 2579-2605, [Visualising Data using t-SNE](http://jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf).

Starting point

- each data item is obtained as a vector $x$ in a _data space_ $X = \mathbb R^d$ of very high dimension (e.g., more than one million when considering pictures taken through a camera with megapixel resolution),
- existence of a significantly lower intrinsic dimension (e.g., identical subject and pose, but the lens is shifted/rotated), thus each item is described more succintly as a vector $y$ in a _map space_ $Y = \mathbb R^m$,
- relationship between $X$ and $Y$: the latter is nonlinearly embedded in the former in a complex way.

Target: apply _nonlinear dimensionality reduction_ techniques (aka _manifold learning_) in order to map data from $X$ to $Y$

Why?

- exploration / visualization,
- pre-processing step before application of standard statistical techniques or machine learning algorithms.

Simple approaches:

- _feature elimination_ (remove uninformative features, use whatever is left)
- _feature selection_ (the other way around)

We'll deal with the more complex case of _feautre extraction_, where (a few) new features are created as a combination of the original ones.

## PCA

Principal Component Analysis or PCA is a feature extracting technique performing dimensionality reduction through a linear transformation between the data and the map space. This transformation is chosen so as to preserve the information retained by the mapped points, where the amount of information is measured through the variance of data along the various dimensions.

Suppose that, after normalization, the $N$ data points are stacked in order to form a matrix $M$, where each row corresponds to a point and thus $M$ has $N$ rows and $d$ columns. The matrix $M^\top M$ is therefore square and has $d$ rows and $d$ columns, and each of its elements is related to the variances and covariances of data along the various dimensions. Denote by $\lambda_1, \dots, \lambda_d$ all eigenvalues of $M^\top M$, sorted in nonincreasing absolute value, and by $e_1, \dots, e_d$ the corresponding eigenvectors. Arranging these vectors from left to right as columns, we obtain a matrix $E$ that corresponds to a linear application within the data space. This application corresponds to rotating the Euclidean reference system so that the first dimension corresponds to the direction along which there is much higher variance on data, the second dimension corresponds to the next higher component, and so on.

The product $M \cdot E$ is a $N \times d$ matrix whose rows correspond to the image of a data point according to the rotation described in the previous paragraph. Having fixed the map dimension $m$ and letting $k = d-m$, the PCA technique consists in cutting the $k$ righmost columns of $E$ in order to obtain a reduced matrix $E_k$, and then computing $M \cdot E_k$. The resulting matrix has $N$ rows and $m$ columns, and its $i$-th row identifies the corresponding map point $y_i$.

Note that the matrix $A = M^\top M$, which by definition has a considerable number of rows and columns, might be impossible to store in memory; even the computation of the matrix product might pose a problem, especially when $N$ grows. Both issues can be addressed exploiting a distributed computational and storage system, such as Apache Hadoop. Indeed, the map-reduce framework can be used in order to compute the product between two matrices and the product between a matrix and a vector. The latter operation, in turn, is a basic brick for the power method which computes the eigenvector $x_1$ corresponding to the eigenvalue $\lambda_1$ having maximal absolute value (from now on, the largest eigenvector, for sake of conciseness). In order to find the remaining eigenvectors, the following trick can be used: considering (without loss of generality) only unit eigenvectors and having defined $A^* = A - \lambda_1 x_1 \cdot x_1^\top$, it can be shown that the largest eigenvector of $A^*$ equals the second largest eigenvector of $A$. Indeed, $x_1$ is also an eigenvector of $A^*$, but now the corresponding eigenvalue nullifies:

$$ A^* x_1 = \left( A - \lambda_1 x_1 \cdot x_1^\top \right) x_1
     = A x_1 - \lambda_1 x_1 \left( x_1^\top x_1 \right)
     = A x_1 - \lambda_1 x_1 ||x_1||^2
     = \lambda_1 x_1 - \lambda_1 x_1 = 0,$$
    
Moreover, any of the remaining eigenvectors $x$ of $A$ is also an eigenvector of $A^*$, and its eigenvalue $\lambda$ remains unchanged:

$$ A^* x = \left( A - \lambda_1 x_1 \cdot x_1^\top \right) x
     = A x - \lambda_1 x_1 (x_1^\top x)
     = A x - \lambda_1 x_1 0 = A x = \lambda x,
$$

where in the last equation $x_1^\top x$ nullifies because $x_1$ and $x$, both being eigenvectors of $A$, are normal.

Summing up, once $x_1$ and $\lambda_1$ have been computed, it is possible to apply the power method to $A^*$ and obtain $x_2$ and $\lambda_2$. After $d$ iterations, all eigenvalues are available. Note that the process of vector normalization is advisable also at each single step of the power method, in order to avoid overflow errors (recall indeed that for each initial vector $v$ we have that $A^k v$ tends to identify with $\lambda_1^k x_1$, thus if $\lambda_1 > 1$ the method will diverge).

We will use the MNIST handwritten digits dataset as a running example for all the proposed techniques. This dataset contains around 60K images, each encoded as a 28x28 array of grayscale values and coupled with a label describing the corresponding digit. More precisely, we will use the `reshape` and `one_hot` options here below in order to:

- reshape each data item as a flattened vector (thus $d = 28 \cdot 28 = 786$), and
- describe labels as numbers in $\{0, 1, \dots, 9\}$.

In [ ]:
import tensorflow as tf
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In order to reduce the computation time and avoid memory issues, we will only use the first 1,000 images in the dataset. We will also sort them in function of the corresponding digit. This is not necessary _per se_, although it will enable us to use a specific visualization technique which we will elaborate on in the following.

In [ ]:
import numpy as np

num_digits = 1000     # change this value in order to process more data (and increase the computation time)

X = np.array([d.reshape([28*28]) for d in x_train[:num_digits]])
y = y_train[:num_digits]

X = np.vstack([X[y==i] for i in range(10)])
y = np.hstack([y[y==i] for i in range(10)])

It is not unlikely that some digit might not be represented in our selection, and this would break some of the forthcoming computations. We can check that everything went fine using a simple assertion.

In [ ]:
# will raise an AssertionError if at least one digit
# is not represented in the subset we selected

assert(list(np.unique(y)) == list(range(10)))

We will refer to the PCA implementation within the `sklearn` library. As a first attempt, we will compute the function which extracts 30 principal components (thus $m=30$ in our previously introduced notation). This process is done following a general pattern used in sklearn: first, an instance of the used algorithm is created (in this case, using the `PCA` constructor and passing as argument the number of components to be considered), and subsequently invoke on this instance the method `fit` having as argument the original data.

In [ ]:
import time
from sklearn.decomposition import PCA

num_components = 30

time_start = time.time()
pca = PCA(n_components=num_components)
pca.fit(X)
print('PCA with 30 components done!'
      ' Time elapsed: {:.2f} seconds'.format(time.time()-time_start))

Note that the processing time is very low (although a better time measurement technique, such as using the `%timeit` magic, would yield a more accurate result). We can now apply the obtained function to the original data in `X`, in order to obtain the corresponding lower-dimensional representations:

In [ ]:
X_pca30 = pca.transform(X)
X_pca30.shape

Note also that, as we are building the PCA mapping using the same data which we also want to transform, we could have directly used the method `fit_transform`.

Having extracted 30 components makes up a compression rate of $30 / 786$, that is

In [ ]:
30/786

and thus we reduced our data to slightly less than 4% of the starting size. In order to verify if the compressed data are still able to meaningfully represent the original ones, we can verify how much of the total variance has been retained. This can be done accessing the `explained_variance_ratio_` field of the `PCA` class, which returns a vector whose components are the percentage of variance explained by each of the extracted features, expressed as a float number between 0 and 1. Plotting the cumulative sum of such percentages we obtain the following picture, showing that 30 components correspond to more than 70% of the original variance.

In [ ]:
# We'll use matplotlib for graphics.
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
%matplotlib inline

# We import seaborn to make nice plots.
import seaborn as sns
sns.set_style('darkgrid')
sns.set_palette('muted')
sns.set_context("notebook", font_scale=1.5,
                rc={"lines.linewidth": 2.5})

plt.plot(range(num_components), pca.explained_variance_ratio_.cumsum())
plt.show()

In [ ]:
sum(pca.explained_variance_ratio_)

Although a compression rate of 4% is remarkable if compared with a retained information of 70%, 30 features are far too big for a visualization-based exploratory analysis. In order to show results in a picture, a much smaller number of components should be extracted: namely, two or three. If we stick for sake of simplicity and readability to the first option, we have that the two bigger principal components retain a total of 17.68% of information:

In [ ]:
sum(pca.explained_variance_ratio_[:2])

In spite of this not particularly encouraging result, let's see what happens when we extract two principal components and use them to produce a scatter plot.

In [ ]:
time_start = time.time()
pca = PCA(n_components=2)
X_pca2 = pca.fit_transform(X)
print('PCA with two components done! Time elapsed: {:.2f} seconds'.format(time.time()-time_start))

In [ ]:
X_pca2[:5]

As we will produce different scatter plots in order to compare various dimensionality reduction strategies, we can define the following `scatter` function which shows each mapped point colored according to the digit it represents, and subsequently positioning each of the ten digits in the median of the corresponding mapped points.

In [ ]:
def scatter(x, labels):
    # We choose a color palette with seaborn.
    palette = np.array(sns.color_palette("hls", 10))

    # We create a scatter plot.
    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], 
                    c=palette[labels],
                    alpha=0.5)
    
    plt.xlim(-25, 25)    # we fix X and Y range
    plt.ylim(-25, 25)    # (-25, 25) hardcoded after inspection
    ax.axis('off')       # we don't actually need axes
    ax.axis('tight')     # this avoids a cluttered graph

    # We add the labels for each digit.
    txts = []
    for i in range(10):
        # Position each label on the median of the corresponding mapped points.
        if np.sum([labels==i]):
            xtext, ytext = np.median(x[labels == i, :], axis=0)
            txt = ax.text(xtext, ytext, str(i), fontsize=24)
            txt.set_path_effects([
                PathEffects.Stroke(linewidth=5, foreground="w"),
                PathEffects.Normal()])
            txts.append(txt)

    return f, ax, sc, txts

The result for the extracted principal components is the following.

In [ ]:
fig_pca, ax_pca, _, _ = scatter(X_pca2, y)
plt.show()

The result is interesting: despite the low retained amount of information, some digits (for instance 0 and 1) are fairly well separated:

In [ ]:
def extract(X, y, i1, i2):
    X_proj = np.vstack([X[(y==i1,)], X[(y==i2,)]])
    y_proj = np.hstack([y[(y==i1,)], y[(y==i2,)]])
    return X_proj, y_proj

_ = scatter(*extract(X_pca2, y, 0, 1))
plt.show()

However, there are other digits, such as 7 and 9, whose mapped points are highly overlapped:

In [ ]:
_ = scatter(*extract(X_pca2, y, 7, 9))
plt.show()

Besides the inherent linear bejaviour of the found mapping, the relatively poor behaviour of PCA is due to the fact that it tends to transform distant data points onto images which also are distant within the map space. This fact, indeed, does not also guarantee that the images of points which are close in the data space are themselves close in the map space. However, this technique has some advantages which should be taken into account, such as its low running time and the fact that it is based on a deterministic procedure. In the next section we will see that it is possible to work with nonlinear mappings which bring more accurate results in separating the low-dimensional representation of digits. This comes at the price of a considerably higher computation time, as well as of relying on a nondeterministic algorithm, thus several executions on the same datasets will tipycally yield different results.

## t-SNE

As seen in the previous section, PCA tends to map distant (that is, dissimilar) points into distant low-dimensional representation. Instead, the t-SNE (t-distributed Stochastic Neighbor Embedding) technique is explicitly driven by the aim of mapping similar (that is, close) points into close low-dimensional representations.

Before deeping into its mathematical details, let's see how it behaves on the same dataset of MNIST digits. Its implementation in sklearn follows the same interface which we have seen for the PCA technique, thus it is sufficient to create an instance of the `TSNE` class and invoke on it the method `fit_trasform`.

In [ ]:
import datetime
import math
from sklearn.manifold import TSNE

rs = 20190105

# this is just to print time values in a more human readable form
def to_hhmmss(seconds):
    times = list(map(int,
                     str(datetime.timedelta(seconds=math.floor(seconds))).split(':')))
    units = ('hour', 'minute', 'second')
    plurals = ['s' if int(t) else '' for t in times]

    return ' '.join(['{} {}{}'.format(t, u, p)
                    for t, u, p in zip(times, units, plurals) if t])

time_start = time.time()
X_tsne = TSNE(random_state=rs).fit_transform(X)    # use the optional argument random_state in order to set the random seed
elapsed = to_hhmmss(time.time()-time_start)
print(f't-SNE with two components done! Time elapsed: {elapsed}')

As anticipated, the running time of the procedure is considerably higher. We can now see how map points distribute in the low-dimensional space, using the `scatter` function in order to produce a result comparable to that obtained with PCA.

In [ ]:
fig_tsne, ax_tsne, _, _ = scatter(X_tsne, y)
plt.show()

As already pointed out, the t-SNE technique is based on a stochastic procedure, thus the graph you will obtain might be different from this one if you don't set the `random_state` optional argument. The result is for sure better than PCA in terms of a good separation of the various digits.

The mathematical formulation of t-SNE is heavily based on the concept of _similarity_ between points. This concept il modeled in a sligtly different way in data and map space. Concerning data space, and under the hypothesis that each point $x_i$ is the mean of a Gaussian distribution having $\sigma_i$ as standard deviation, the starting point is a measurement of the conditional probability that $x_i$ will choose $x_j$ as its neighbor, under the assumption that this choice is weighted according to the densities of the above mentioned distributions. This leads to the probability $p_{j|i}$ defined as follows:

\begin{equation}
p_{j|i}=\frac{\exp\left( âˆ’ \frac{ ||x_iâˆ’x_j||^2}{2\sigma^2_i} \right)}{\sum_{k\neq i}\exp \left(âˆ’ \frac{ ||x_iâˆ’x_k||^2}{2\sigma^2_i} \right)}
(1 - \delta_{ij}),
\end{equation}

where $\delta_{ij}$ denotes the Kroneker delta (that is, it evaluates to 1 when $i = j$ and to 0 otherwise). This coherently implies that $p_{i|i} = 0$, as we are modeling the similarity between pairs of different items. For instance, in the following simple experiment ten points are considered in $\mathbb R$, together with the corresponding Gaussian distributions having unit variance. The obtained graphs show how $p_{j|5}$ changes when the standard deviation associated to the fifth point is set to 0.1, 1, and 2, respectively.

In [ ]:
xs = np.arange(10)
sigmas = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

def density(x_i, x_j, sigma_i):
    return math.exp(-(x_i - x_j)**2 / (2* sigma_i**2))

def p_similar(j, i):
    return density(xs[i], xs[j],
                   sigmas[i]) / sum([density(xs[i], xs[k], sigmas[i])
                                     for k in range(10) if k != i])

def get_prob_graph(sigma_4, ax):
    sigmas[4] = sigma_4
    x_vals = [x for x in xs if x != 4]
    p_vals = [p_similar(x, 4) for x in x_vals]
    ax.vlines(x_vals, 0, p_vals)
    ax.set_title(r'$\sigma = {}$'.format(sigma_4))

f, axs = plt.subplots(1, 3, figsize=(15,3))
get_prob_graph(.1, axs[0])
get_prob_graph(1, axs[1])
get_prob_graph(2, axs[2])
plt.show()

Although we cannot use $p_{j|i}$ directly as a similarity measure, because it lacks the symmetry property, it is rather easy to obtain its symmetrization $p_{ij}$ as

$$
p_{ij} = \frac{p_{j|i} + p_{i|j}}{2N},
$$

and we can organize all $p_{ij}$s into a _similarity matrix_ $P$. We can build this matrix in function of the _pairwise distances_ $|| x_i - x_j ||^2$, which in turn can be organized in a matrix; actually, sklearn provides a `pairwise_distances` method which does this for us. It is worth visualizing the matrix, or rather a smaller subset of its elements (in order to avoid a long computation time, as this matrix would contain $10^8$ elements). More precisely, we will consider only one tenth of rows, evenly spaced. Idem for the columns.

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances
pal = sns.light_palette("blue", as_cmap=True)

D = pairwise_distances(X, squared=True)
plt.imshow(D[::1, ::1], interpolation='none', cmap=pal)
plt.axis('off')
plt.title(r'Distance matrix', fontdict={'fontsize': 16})
plt.show()

In this visualization a lighter shade of blue corresponds to lower distance values, whereas bigger distances are marked with a darker shade. The image highlights some sort of structure between the data: indeed, there are sevral light colored squares along the diagonal. This is due to the fact that data was sorted according to the labels, thus each square likely corresponds to blocks of items having the same label, that is images of a same digits which we would expect to be similar in terms of distances.

The visualization of the similarity matrix would require to provide also the values for all standard deviations $\sigma_i$s. Leaving this aspect aside for a moment, we can easily apply the previously introduced definition for, say, $p_{j|i}$ directly to the distance matrix and show the result.

In [ ]:
def _joint_probabilities_constant_sigma(D, sigma):
    P = np.exp(-D**2/(2 * sigma**2))
    P -= np.diag(P.diagonal())
    P /= np.sum(P, axis=1)    # here we get p_j|i
    P *= 2/D.shape[0]         # here we get p_ij in force of symmetricity and constant sigma
    return P

In [ ]:
P_constant = _joint_probabilities_constant_sigma(D, 6**7)

plt.imshow(P_constant[::1, ::1], interpolation='none', cmap=pal)
plt.axis('off')
plt.title(r'$p_{j|i}$ (constant $\sigma$)', fontdict={'fontsize': 16})
plt.show()

To get the actual similarity matrix it is necessary to also fix the values for standard deviations. Actually, it is not required to directly provide them, as t-SNE automatically estimates them in function of a user-tunable parameter called _perplexity_. Perplexity is defined in terms of entropy of the Gaussian distributions centered around data points, and it is a smooth proxy for number of neighbors to be considered. We can directly use the `_joint_probabilities` method implemented in sklearn in order to show the similarity matrix actually used by t-SNE.

In [ ]:
from sklearn.manifold._t_sne import _joint_probabilities
from scipy.spatial.distance import squareform

# Similarity with variable sigma.
P_binary = _joint_probabilities(D, 6.**7, False)
# The output of this function needs to be reshaped to a square matrix.
P_binary_s = squareform(P_binary)

plt.imshow(P_binary_s[::10, ::10], interpolation='none', cmap=pal)
plt.axis('off')
plt.title(r'$p_{j|i}$ (variable $\sigma$)', fontdict={'fontsize': 16})
plt.show()

The concept of similarity is modeled in a similar way in the map space, although with a peculiar difference w.r.t. data space. Here, each map point is associated to a _Cauchy distribution_ (a special case we obtain when fixing to one the number of degrees of freedom in the Student's t-distribution). This induces a similarity matrix $Q$ whose generic element has the following analytical form:

$$
q_{ij} = \frac{\left( 1 + \left|\left| y_i - y_j \right|\right|^2 \right)^{-1}}{\sum_{k \neq i} \left( 1 + \left|\left| y_i - y_k \right|\right|^2 \right)^{-1}}.
$$

An important thing to note is that both $P$ and $Q$ (the similarity matrices in data and map space, respectively) can also be interpreted as probability distributions. Namely, each $p_{ij}$ is the joint probability to get the pair $(x_i, x_j)$ if we randomly select one data point $x_i$ and its neighbour $x_j$. Idem for $q_{ij}$ and points in the map space. The t-SNE algorithm chooses the map points in such a way that the distributions $P$ and $Q$ are as close as possible. The used tool to measure closeness between probability distributions is the _Kullback-Leibler_ divergence, which in this case reads

$$
\mathrm{KL}(y_1, \dots, y_N) = \sum_{i, j} p_{ij} \log \frac{p_{ij}}{q_{ij}}.
$$

Note that $\mathrm{KL}$ only depends on $y_1, \dots, y_N$, because $P$ is completely known (i.e., each $p_{ij}$ is a number), while $Q$ depends on the choice of the map points. The latter are precisely chosen in such a way that $\mathrm{KL}$ gets minimized. The associated optimization process exploits a sophisticated variant of the gradient descent algorithm, starting from the fact that

$$
\frac{\partial \mathrm{KL}(y_1, \dots, y_N)}{\partial y_i} =
4 \sum_j (p_{ij} - q_{ij})(y_i - y_j)\left( 1 + \left|\left| y_i - y_j \right|\right|^2 \right)^{-1}.
$$

The following cell redefines the sklearn implementation of this gradient descent algorithm, saving at each iteration the current position of all map points. Subsequently, t-SNE is executed afresh on the MNIST digits dataset.

In [ ]:
import sklearn
from numpy import linalg

_original_gradient_descent = sklearn.manifold._t_sne._gradient_descent


# This list will contain the positions of the map points at every iteration.
positions = []

def _gradient_descent(objective, p0, it, n_iter,
                      n_iter_check=1,
                      n_iter_without_progress=300,
                      momentum=0.5, learning_rate=1000.0, min_gain=0.01,
                      min_grad_norm=1e-7, min_error_diff=1e-7, verbose=0,
                      args=[], kwargs={}):
    # The documentation of this function can be found in scikit-learn's code.

    p = p0.copy().ravel()
    update = np.zeros_like(p)
    gains = np.ones_like(p)
    error = np.finfo(np.float).max
    best_error = np.finfo(np.float).max
    best_iter = 0

    for i in range(it, n_iter):
        # We save the current position.
        positions.append(p.copy())

        new_error, grad = objective(p, *args)
        error_diff = np.abs(new_error - error)
        error = new_error
        grad_norm = linalg.norm(grad)

        if error < best_error:
            best_error = error
            best_iter = i
        elif i - best_iter > n_iter_without_progress:
            break
        if min_grad_norm >= grad_norm:
            break
        if min_error_diff >= error_diff:
            break

        inc = update * grad >= 0.0
        dec = np.invert(inc)
        gains[inc] += 0.05
        gains[dec] *= 0.95
        np.clip(gains, min_gain, np.inf)
        grad *= gains
        update = momentum * update - learning_rate * grad
        p += update

    return p, error, i

# temporarily modify gradient descent optimization
sklearn.manifold._t_sne._gradient_descent = _gradient_descent

X_proj = TSNE().fit_transform(X)

# get back to official implementation
sklearn.manifold._t_sne._gradient_descent = _original_gradient_descent

Now we can rearrange all map point positions at each iteration and create a 3D array in which the added dimension can be interpreted as time.

In [ ]:
X_iter = np.dstack(position.reshape(-1, 2)
                   for position in positions)

In the end, we are able to create an animation showing how the map points converge to their final position as t-SNE evolves its optimization of the Kullback-Leibler divergence between data and map space.

In [ ]:
# We'll generate an animation with matplotlib and moviepy.
from moviepy.video.io.bindings import mplfig_to_npimage
import moviepy.editor as mpy

f, ax, sc, txts = scatter(X_iter[..., -1], y)
plt.close()

def make_frame_mpl(t):
    i = int(t*40)
    x = X_iter[..., i]
    sc.set_offsets(x)
    for j, txt in zip(range(10), txts):
        xtext, ytext = np.median(x[y == j, :], axis=0)
        txt.set_x(xtext)
        txt.set_y(ytext)
    return mplfig_to_npimage(f)

animation = mpy.VideoClip(make_frame_mpl,
                          duration=X_iter.shape[2]/40.)

animation.ipython_display(fps=20, width=280)

The animation highlights different behaviours in the optimization phase:

- at the beginning of the process map points are forced to globally lay at small distances one another; this trick, called _early compression_, allows a more efficient exploration of the different ways to organize data, as it is easier for one point to move from one cluster to another one when the latter are not distant;
- in addition, all $p_{ij}$ values are initially magnified through multiplication by a factor of, say, 4; this has the effect of promoting large values also for $q_{ij}$s, which in turn tends to create widely separated clusters, so there is "empty" space allowing these clusters more degrees of freedom when moving in their quest for a global organization in the low-dimenstional space;
- in the final stages of the process the clusters assume a less crowded shape and move towards the optimal positions.

We can also repeat the previous process focusing now on how the similarity matrix $Q$ in map space changes as the algorithm moves toward its final output. Here we can see the matrix-based counterpart of the optimization phases.

In [ ]:
from scipy.spatial.distance import pdist

n = 1. / (pdist(X_iter[..., -1], "sqeuclidean") + 1)
Q = n / (2.0 * np.sum(n))
Q = squareform(Q)

f = plt.figure(figsize=(6, 6))
ax = plt.subplot(aspect='equal')
im = ax.imshow(Q, interpolation='none', cmap=pal)
plt.axis('tight')
plt.axis('off')
plt.close()


def make_frame_mpl(t):
    i = int(t*40)
    n = 1. / (pdist(X_iter[..., i], "sqeuclidean") + 1)
    Q = n / (2.0 * np.sum(n))
    Q = squareform(Q)
    im.set_data(Q)
    return mplfig_to_npimage(f)

animation = mpy.VideoClip(make_frame_mpl,
                          duration=X_iter.shape[2]/40.)
animation.ipython_display(fps=20, width=280)

The use of a Cauchy distribution in the low-dimensional space is devoted to avoiding one of the problems typical of the so-called _curse of dimensionality_. Namely, as the dimension of a space gets higher, points picked at random in a sphere $S$ centered in the origin will tend to concentrate nearer and nearer the surface of $S$. This is exemplified in the next simulation, computing and showing the histograms of distances from the origin of one thousand points drawn in the unit sphere, and considering data spaces of dimensions 2, 5, and 10, respectively.

In [ ]:
npoints = 1000
plt.figure(figsize=(15, 4))
for i, D in enumerate((2, 5, 10)):
    # Normally distributed points.
    u = np.random.randn(npoints, D)
    # Now on the sphere.
    u /= linalg.norm(u, axis=1)[:, None]
    # Uniform radius.
    r = np.random.rand(npoints, 1)
    # Uniformly within the ball.
    points = u * r**(1./D)
    # Plot.
    ax = plt.subplot(1, 3, i+1)
    ax.set_xlabel('Ball radius')
    if i == 0:
        ax.set_ylabel('Distance from origin')
    ax.hist(linalg.norm(points, axis=1),
            bins=np.linspace(0., 1., 50))
    ax.set_title(r'$D={}$'.format(D), loc='left')
plt.show()

Thus pairs of points in the high-dimensional starting place will often have a big distance, but the volume available in the low-dimensional space is far smaller. Using also in this space Gaussian distributions, whose density tends to vanish in correspondence of high values, might lead to unsatisfactory mappings with map points too close one another (the so-called _crowding problem_). The t-SNE algorithm fights this problem using the Cauchy distribution, whose density

$$f_\mathrm C (x) = \frac{1}{\pi} \frac{1}{1 + x^2} $$

has a bell shape not dissimilar to that of the Gaussian distribution, although with heavier tails, as highlighted by the following figure.

In [ ]:
from scipy.stats import cauchy, norm

z = np.linspace(-5., 5., 1000)
plt.plot(z, norm.pdf(z), label='Normal')
plt.plot(z, cauchy.pdf(z), label='Cauchy')
plt.legend()
plt.show()

## More control on t-SNE parameters

The implementation of t-SNE provided by sklearn allow finer control on the algorithm by tuning the following hyperparameters:

- `n_components`,
- `perplexity`,
- `early_exaggeration`,
- `learning_rate`,
- `n_iter`,
- `method`.

Each of them, corresponding to an optional argument of the `TSNE` class constructor, is briefly described here below.


### Dimension of the map space
The `n_components` argument allows to explicitly set the dimension $m$ of the map space. In spite of the possibility of using arbitrary values, when applying t-SNE for visualization purposes only a few choiches are available (unless we resort to sophisticated visualization techniques). In our experiments we have used the default value, which maps data points in $\mathbb R^2$.

### Perplexity
The `perplexity` argument allows to control the perplexity parameter initially used in order to estimate the standard deviations of distributions in data space. Its default value is 30, and it is advisable to tune it in the $[5, 50]$ interval.

### Early exaggeration
The higher this value, defaulting to 12.0, the more space will be promoted between clusters during the early exaggeration phase.

### Learning rate
In t-SNE the learning rate typically assumes higher values for the learning rate than with typical machine learning algorithm based on gradient descent. Its default value is 200.0, and it can be modified through the `learning_rate` argument; the sklearn documentation suggest to consider values in the range $(10.0, 1000.0)$.

### Number of iterations
The number of iterations of the gradient descent optimization is ruled by the `n_iter` argument. Its default value is 1000, and the documentation advises to avoid considering less than 250 iterations.

### Approximated vs. exact execution
When the argument `method` is set to `'exact'`, t-SNE is executed exactly and it requires an execution times of $\mathrm O(N^2)$. The value `'barnes_hut'` uses the Barnes-Hut heuristic that lowers the execution time to $\mathrm O(N \log N)$. This speeds up the computation time at the price of an approximated result, and it is the only option available when millions of data need to be processed. Note that this second option is also the default one. In the literature other similar heuristics have been proposed, for instance exploiting a random walk over a graph connecting neighbors selected by the original algorithm when data is undersampled.

## Jointly using PCA and t-SNE

It is advisable to not directly use t-SNE on the available data, especially when the data dimension $D$ is very high. In such chases it is indeed advisable to start by a preprocessing phase aimed at reducing $D$ to, say, 30, through a less computationally expensive procedure. The intermediate result can be subsequently fed to t-SNE with the advantage of getting rid of noise in data and to speed up the computation of distances in the data space. PCA is an ideal candidate for this preprocessing phase.

We can very easily adapt the code already written to this two-step dimensionality reduction procedure, as we already have compressed the digits data to 30 dimensions using PCA, thus the only things to be done is to fed the result of such compression to t-SNE and graphically show the result.

In [ ]:
time_start = time.time()
X_tsne = TSNE(random_state=rs).fit_transform(X_pca30)
print('t-SNE + PCA done! Time elapsed: {} seconds'.format(time.time()-time_start))

fig_snepca, ax_snepca, _, _= scatter(X_tsne, y)
plt.show()

## An example: word2vec

The term _word embedding_ denotes an operation widely used in the realm of _natural language processing_ (NLP for short) whose aim is that of converting words in numerical form. A word embedding technique which has recetnly gained a lot of attention, known as _word2vec_, allows us to show another application of t-SNE.

Word2vec uses a neural network in order to both

- compress a digital encoding for a set of words, and
- associate the compressed encodings to a _meaning_ of the words within a specified context.

Such context is provided as a _corpus_ of documents. More precisely, starting from this corpus it is easy to build a _one-hot_ encoding for all occurring words, which means that each is associated to a vector whose component all are set to zero, except one which set to 1. This is done taking care of mapping different words to different vectors, with the obvious flaw of obtaining vectors of a potentially very high dimension, as the latter will coincide with the number of different words in the corpus. Moreover, such vectors do not have a meaning attached to them, as they only depend on the order in which words in the corpus have been enumerated.

Although this initial encoding is not efficiently usable in practice, it can be used in order to train a neural network in the following way: let $N$ denote the number of different words in the considered corpus, and _a fortiori_ the dimension of one-hot encodings, and consider a feed-forward neural network having $N$ inputs, $M < N$ hidden neurons, and $N$ outputs. Given a word $w$, it is relatively easy to estimate the probability $p(w, w')$ that $w$ is contained in the same n-gram of another word $w'$ (here $n$-grams are identified by a set of $n$ words occurring in sequence in at least a document in the corpus, and $n$ is fixed beforehand). Therefore, it is possible to associate each word $w$ to

- an _input vector_ of $n$ components coinciding with the one-hot encoding of $w$ itself, and
- an _output vctor_ whose $n$ components are the probabilities $p(w, w')$ for all the possible choice of $w'$.

Using the corresponding training set it is thus possible to train the previously described neural network. Once this has been done, the hidden layer identifies with a compression of the initial encoding of words. Indeed, the word2vec encoding is finally computed by taking all words, map their one-hot enconded vectors to the hidden layer, and consider the obtained vecors of $M$ elments. These vectors typically have the nice property that two _similar_ words are mapped to _close_ vectors according to the cosine distance (this is the above mentioned meaning attached to the final encoding).

A more thorough explaination of word2vec is available at https://adventuresinmachinelearning.com/word2vec-tutorial-tensorflow/.

Although word2vec is itself a sort of compression technique, it works when the corresponding map space dimension $M$ is on the order of some hundred. Thus it is not possible to graphically show the obtained result. The main property of t-SNE, namely that of preserving the closeness of vectors, suggests thus to use this technique in order to further reduce the dimension of the encoded words from $M$ to 2. In order to do this we will start from an already trained word2vec encoding, available for download.

In [ ]:
import os, urllib

url = 'https://media.githubusercontent.com/media/eyaler/word2vec-slim/master/'
filename = 'GoogleNews-vectors-negative300-SLIM.bin.gz'
    
if not os.path.exists(filename):
        filename, _ = urllib.request.urlretrieve(url + filename, filename)

Note that the download process will likely take a considerable time, as the file to be downloaded is 260 Mb big. This is rather a small dimension for a word2vec encoding, because obtaining a good result is related to processing a rich corpus of documents. Indeed, the typical size of a trained word2vec model is of the order of magnitude of Gbytes. For sake of simplicity, we have considered here a _slim_ model build from a subset of documents taken from Google news.

The serialized version of the pretrained word2vec model contained in the downloaded file can be accessed without unzipping it through the use of the `gensim` module.

In [ ]:
import gensim

model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300-SLIM.bin.gz', binary=True)

The result is a python dictionray having words as key and the corresponding word2vec vectors as values. As specified in the file name, each vector has 300 elements:

In [ ]:
len(model['computer'])

Just in order to see what an encoded vector looks like, we can visualize the first ten components of the encoding of "computer":

In [ ]:
model['computer'][:10]

The model itself is equipped with a `similar_by_word` which automatically retrieves the terms more similar to a specified word (here, the `topn` argument specifies how many such terms will be retrieved):

In [ ]:
model.similar_by_word('computer', topn=7)

Note that each of the retrieved terms is coupled with a floating point value, which is the cosine distance between each term and the starting word. Let's define a `word_plot` function which retrieves the most similar terms and applies t-SNE to the corresponding word2vec vectors, so as to be able to show a scatter plot of the result, labeled with the similar terms.

In [ ]:
def word_plot(word, topn=10):

    similar_words = [w for w, i in model.similar_by_word(word, topn=topn)]
    data_points = [model[w] for w in similar_words]

    tsne = TSNE(random_state=20190104)
    map_points = tsne.fit_transform(data_points)
    x_coords = map_points[:, 0]
    y_coords = map_points[:, 1]
    plt.scatter(x_coords, y_coords, alpha=0.8)

    for w, x, y in zip(similar_words, x_coords, y_coords):
        plt.annotate(w, xy=(x, y), xytext=(0, -10),
                     textcoords='offset points',
                     ha='center', va='top')
    plt.axis('off')
        
    plt.show()

For instance, starting by the word "computer" we obtain the following graph:

In [ ]:
word_plot('computer', 15)

Note how one of the retrieved terms is "laptop" (which is perfectly fine), but if we start from the latter term we obtain a totally different graph.

In [ ]:
word_plot('laptop')

In [ ]:
import random

def word_trajectory(word, n, by, verbose=False):
    word_traj = [word]
    vect_traj = [model[word]]
    for _ in range(n):
        words = model.similar_by_word(word, topn=by)
        while True:
            word = words[random.randint(0, by-1)][0]
            if word not in word_traj:
                break
        if verbose:
            print(word)
        word_traj.append(word)
        vect_traj.append(model[word])
    return word_traj, vect_traj

In [ ]:
res = word_trajectory('computer', 10, 40, verbose=True)

In [ ]:
tsne = TSNE(random_state=20190104)
map_points = tsne.fit_transform(res[1])

In [ ]:
x_coords = map_points[:, 0]
y_coords = map_points[:, 1]

In [ ]:
plt.scatter(x_coords, y_coords, alpha=0.8)

for i in range(9):
    plt.arrow(x_coords[i], y_coords[i], x_coords[i+1]-x_coords[i], y_coords[i+1]-y_coords[i],
              head_width=10, head_length=10, fc='k', ec='k')

for w, x, y in zip(res[0], x_coords, y_coords):
    plt.annotate(w, xy=(x, y), xytext=(0, -10),
                 textcoords='offset points',
                 ha='center', va='top')
plt.show()